In [1]:
import argparse
import math
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
import configparser
#import grequests # for async requests, conflicts with requests somehow
import requests
import numpy as np
from numpy import genfromtxt
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import morton


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def F0(cubes):
    f0 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    #f0 = np.sum(cube)
    return(f0)
    
def F1(cubes):
    s = cubes.shape

    f1 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    out = f1

    return(out)



In [5]:
def getCube(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    print(di['loc']) #DEBUG
    sys.stdout.flush() #DEBUG
    ### OUTPUT will be a numpy array in Z Y X order 
    avg = np.mean(data)
    return(avg)


In [6]:
CONFIG_FILE = "neurodata.conf"
COLL_NAME = "collman"
EXP_NAME = "M247514_Rorb_1_Site3Align2_LENS_Session1_CROP"
COORD_FRAME = COLL_NAME + "_" + EXP_NAME
BF = 5

LOCATIONS = "rorb_locations_at_buffered.csv"

CHAN_NAMES = ['DAPI1', 'DAPI2', 'DAPI3', 'GABA', 'GAD2', 'Gephyrin', 'GluN1',
                'MBP', 'PSD95', 'synapsin', 'TdTomato', 'VGlut1']

num_threads = 4

In [7]:
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)

In [8]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
L

[[230, 186, 38, 0],
 [238, 188, 40, 1],
 [239, 209, 25, 4],
 [240, 241, 10, 5],
 [248, 228, 16, 6],
 [249, 220, 10, 7],
 [256, 218, 17, 8],
 [228, 183, 38, 9],
 [222, 181, 40, 10],
 [221, 184, 34, 11],
 [187, 192, 34, 12],
 [191, 190, 32, 13],
 [199, 193, 34, 14],
 [203, 194, 22, 15],
 [206, 179, 20, 16],
 [205, 188, 15, 17],
 [210, 188, 19, 18],
 [197, 193, 20, 19],
 [196, 194, 17, 20],
 [194, 182, 18, 21],
 [200, 187, 21, 22],
 [222, 183, 23, 23],
 [230, 177, 32, 24],
 [225, 172, 35, 25],
 [222, 163, 38, 26],
 [207, 164, 40, 27],
 [201, 163, 18, 29],
 [208, 163, 34, 31],
 [217, 189, 13, 34],
 [231, 196, 10, 35],
 [225, 178, 14, 36],
 [227, 171, 17, 37],
 [234, 174, 19, 38],
 [235, 167, 16, 39],
 [233, 154, 30, 40],
 [235, 156, 31, 41],
 [241, 169, 16, 42],
 [215, 164, 36, 44],
 [219, 156, 26, 45],
 [233, 160, 29, 46],
 [228, 156, 20, 47],
 [222, 155, 13, 49],
 [223, 161, 21, 50],
 [226, 169, 32, 51],
 [220, 178, 28, 52],
 [213, 184, 12, 53],
 [207, 183, 9, 56],
 [207, 180, 6, 57],
 [

In [12]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
m = morton.Morton(dimensions=3, bits=64)
Lzo = [m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))]
#Lzo = sorted([m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))])
#                                                                                               
ind = np.argsort(Lzo)
loc = [m.unpack(l) for l in sorted(Lzo)]

loc = L
x_rng = [[x[0]-BF, x[0]+BF+1] for x in loc] 
y_rng = [[y[1]-BF, y[1]+BF+1] for y in loc] 
z_rng = [[z[2]-BF, z[2]+BF+1] for z in loc]
ids   = [L[i][3] for i in ind]
ids = [i[3] for i in L]
ids

[0,
 1,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 29,
 31,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 49,
 50,
 51,
 52,
 53,
 56,
 57,
 58,
 59,
 60,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 81,
 83,
 84,
 87,
 88,
 89,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 106,
 109,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 126,
 127,
 128,
 129,
 130,
 131,
 136,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 158,
 159,
 160,
 161,
 162,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 184,
 185,
 186,
 187,
 188,
 189,
 192,
 194,
 195,
 196,
 197,
 200,
 201,
 202,
 203,
 204,
 205,
 207,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 22

In [13]:
ChanList = []
for ch in CHAN_NAMES:
    di = [{
          'rem': rem,
          'ch_rsc':
            ChannelResource(ch,COLL_NAME,EXP_NAME,'image',datatype='uint8'),
          'ch'  : ch,
          'res' : 0,
          'loc' : loc[i],
          'xrng': x_rng[i],  
          'yrng': y_rng[i],  
          'zrng': z_rng[i],  
          'bf'  : BF 
         } for i in range(len(loc))]

    with ThreadPool(num_threads) as tp:
        out = tp.map(getCube, di)
        print(ch) ##DEBUG
        sys.stdout.flush() #DEBUG

    ChanList.append(np.asarray(out))

outArray = np.asarray(ChanList)
## outArray will be a numpy array in [channel, synapse, z, y, x] order
## this is C-ordering


[230, 186, 38, 0][234, 194, 40, 189][194, 195, 7, 60][197, 194, 31, 126]



[238, 188, 40, 1]
[202, 194, 8, 127]
[198, 196, 41, 192][190, 181, 22, 63]

[239, 209, 25, 4]
[225, 176, 39, 194]
[193, 188, 12, 64]
[191, 231, 36, 128]
[203, 182, 10, 65][200, 227, 37, 129]

[222, 178, 33, 195]
[198, 218, 40, 130]
[206, 175, 14, 66]
[215, 181, 35, 196]
[218, 169, 42, 197]
[198, 216, 39, 131]
[205, 161, 32, 67]
[237, 161, 40, 200]
[195, 227, 7, 136]
[193, 171, 13, 68]
[240, 241, 10, 5]
[206, 216, 23, 138]
[237, 166, 34, 201]
[192, 160, 41, 69]
[248, 228, 16, 6]
[205, 216, 11, 139]
[187, 166, 12, 70]
[249, 220, 10, 7]
[192, 224, 20, 140]
[195, 167, 37, 71]
[256, 218, 17, 8]
[198, 223, 10, 141]
[188, 171, 40, 72]
[228, 183, 38, 9]
[197, 179, 26, 73]
[222, 181, 40, 10]
[202, 178, 31, 74]
[221, 184, 34, 11]
[236, 178, 14, 202]
[200, 174, 17, 75]
[187, 192, 34, 12]
[237, 188, 14, 203]
[196, 170, 34, 76]
[191, 190, 32, 13]
[237, 191, 12, 204]
[205, 155, 32, 77]
[199, 193, 34, 14]
[192, 211, 25, 142]


[293, 264, 32, 642]
[292, 228, 40, 521]
[315, 209, 22, 503]
[298, 258, 24, 583]
[292, 226, 30, 522]
[284, 260, 29, 643]
[309, 218, 36, 504]
[306, 260, 38, 584]
[280, 260, 26, 644]
[281, 240, 18, 524]
[310, 218, 32, 505]
[284, 244, 8, 586]
[288, 271, 30, 645]
[282, 241, 13, 525]
[311, 216, 20, 506]
[300, 245, 14, 587]
[292, 286, 24, 646]
[295, 231, 13, 526]
[243, 258, 41, 698]
[290, 255, 13, 588]
[286, 275, 20, 647]
[242, 263, 31, 699]
[309, 227, 29, 527]
[287, 252, 13, 589]
[288, 271, 11, 648]
[307, 226, 24, 528]
[227, 264, 15, 700]
[278, 254, 18, 590]
[297, 266, 14, 649]
[305, 223, 17, 529]
[218, 268, 12, 702]
[311, 254, 14, 591]
[291, 263, 22, 650]
[312, 226, 17, 530]
[220, 261, 13, 703]
[313, 256, 18, 592]
[280, 269, 19, 651]
[220, 261, 21, 704]
[306, 234, 29, 531]
[301, 267, 6, 593]
[284, 282, 30, 652]
[311, 231, 38, 532]
[219, 265, 36, 705]
[315, 265, 32, 594]
[279, 276, 11, 653]
[315, 230, 40, 533]
[219, 266, 41, 706]
[309, 262, 28, 595]
[308, 228, 33, 534]
[286, 278, 14, 654]
[2

[187, 166, 12, 70]
[222, 181, 40, 10]
[192, 224, 20, 140]
[195, 167, 37, 71]
[237, 188, 14, 203]
[221, 184, 34, 11]
[198, 223, 10, 141]
[188, 171, 40, 72]
[237, 191, 12, 204]
[187, 192, 34, 12]
[192, 211, 25, 142]
[197, 179, 26, 73]
[234, 189, 12, 205]
[191, 190, 32, 13]
[197, 213, 24, 143]
[202, 178, 31, 74]
[221, 154, 40, 207]
[199, 193, 34, 14]
[191, 207, 25, 144]
[200, 174, 17, 75]
[217, 265, 40, 209]
[203, 194, 22, 15]
[189, 205, 22, 145]
[196, 274, 37, 210]
[196, 170, 34, 76]
[206, 179, 20, 16]
[189, 208, 16, 146]
[205, 155, 32, 77]
[209, 274, 33, 211]
[205, 188, 15, 17]
[189, 207, 8, 147]
[220, 160, 41, 78]
[244, 262, 38, 212]
[210, 188, 19, 18]
[199, 206, 12, 148]
[204, 159, 42, 81]
[221, 258, 36, 213]
[220, 199, 12, 149]
[197, 193, 20, 19]
[215, 174, 39, 83]
[213, 265, 38, 214]
[241, 207, 34, 150]
[196, 194, 17, 20]
[208, 173, 34, 84]
[210, 263, 33, 215]
[194, 182, 18, 21]
[236, 210, 38, 151]
[227, 163, 26, 87]
[241, 248, 43, 217]
[200, 187, 21, 22]
[237, 207, 43, 152]
[214, 1

[278, 254, 18, 590]
[211, 281, 28, 710]
[309, 227, 29, 527]
[291, 263, 22, 650]
[311, 254, 14, 591]
[209, 284, 36, 711]
[307, 226, 24, 528]
[280, 269, 19, 651]
[313, 256, 18, 592]
[214, 282, 18, 712]
[284, 282, 30, 652]
[301, 267, 6, 593]
[219, 286, 8, 713]
[305, 223, 17, 529]
[279, 276, 11, 653]
[315, 265, 32, 594]
[224, 285, 22, 714]
[312, 226, 17, 530]
[235, 266, 42, 716]
[309, 262, 28, 595]
[286, 278, 14, 654]
[306, 234, 29, 531]
[243, 275, 10, 717]
[317, 264, 20, 596]
[280, 284, 11, 655]
[251, 285, 6, 718]
[311, 231, 38, 532]
[309, 263, 20, 597]
[274, 283, 18, 656]
[253, 285, 8, 719]
[315, 230, 40, 533]
[316, 260, 12, 598]
[258, 268, 17, 659]
[250, 278, 26, 720]
[308, 228, 33, 534]
[304, 272, 38, 600]
[272, 260, 36, 660]
[252, 262, 18, 721]
[301, 225, 10, 535]
[307, 267, 42, 602]
[272, 266, 29, 661]
[289, 239, 32, 536]
[228, 263, 20, 722]
[310, 267, 21, 605]
[269, 265, 28, 662]
[292, 232, 41, 539]
[308, 278, 41, 606]
[221, 267, 28, 723]
[273, 267, 34, 665]
[303, 239, 36, 541]
[222

[221, 258, 36, 213][199, 206, 12, 148]

[204, 159, 42, 81]
[197, 193, 20, 19]
[215, 174, 39, 83]
[220, 199, 12, 149]
[213, 265, 38, 214]
[196, 194, 17, 20]
[208, 173, 34, 84]
[210, 263, 33, 215]
[241, 207, 34, 150]
[194, 182, 18, 21]
[227, 163, 26, 87]
[236, 210, 38, 151]
[241, 248, 43, 217]
[200, 187, 21, 22]
[214, 169, 24, 88]
[237, 207, 43, 152]
[237, 249, 43, 218]
[222, 183, 23, 23]
[217, 167, 18, 89]
[230, 204, 38, 153]
[233, 236, 26, 219]
[230, 177, 32, 24]
[200, 198, 33, 158]
[209, 168, 11, 91]
[223, 235, 36, 220][225, 172, 35, 25]

[195, 198, 34, 159]
[210, 158, 32, 93]
[216, 236, 10, 221]
[222, 163, 38, 26]
[238, 195, 17, 160]
[205, 158, 27, 94]
[207, 164, 40, 27]
[215, 247, 13, 222]
[201, 156, 22, 95][242, 196, 18, 161]

[216, 233, 9, 223]
[201, 163, 18, 29]
[196, 155, 6, 96]
[236, 196, 15, 162]
[199, 251, 32, 224]
[208, 163, 34, 31]
[197, 157, 15, 97]
[235, 179, 29, 164]
[202, 258, 28, 225]
[217, 189, 13, 34]
[195, 156, 35, 98]
[194, 262, 22, 226]
[203, 187, 12, 166]
[231, 1

...

[299, 167, 30, 848][236, 159, 14, 1008]

[204, 286, 16, 908]
[228, 215, 39, 960]
[194, 275, 6, 910]
[241, 194, 24, 1011]
[302, 165, 43, 849]
[242, 215, 26, 961]
[197, 281, 9, 912]
[293, 249, 7, 1012]
[302, 167, 34, 850]
[236, 217, 26, 962]
[195, 280, 21, 913]
[288, 245, 16, 1013]
[296, 161, 42, 851]
[258, 210, 38, 963]
[200, 277, 28, 914]
[238, 254, 7, 1015]
[265, 166, 35, 852]
[258, 198, 32, 964]
[255, 248, 10, 1016]
[201, 273, 22, 915]
[261, 167, 33, 853]
[255, 212, 34, 965]
[257, 270, 30, 1017]
[202, 273, 31, 916]
[278, 158, 34, 854]
[251, 212, 40, 966]
[200, 252, 27, 1019]
[200, 276, 14, 917]
[276, 173, 32, 855]
[251, 213, 42, 967]
[202, 273, 26, 918]
[211, 249, 40, 1020]
[275, 184, 28, 856]
[251, 203, 30, 968]
[198, 267, 31, 919]
[208, 249, 42, 1021]
[275, 183, 20, 857]
[255, 202, 35, 969]
[193, 266, 23, 920]
[190, 253, 42, 1022]
[275, 178, 22, 858]
[254, 200, 32, 970]
[202, 264, 38, 1026]
[196, 279, 29, 921]
[266, 182, 21, 859]
[250, 207, 10, 972]
[225, 277, 19, 1027]
[188, 279, 

[309, 208, 30, 485]
[284, 231, 10, 360]
[279, 185, 35, 426]
[230, 221, 37, 292]
[300, 215, 42, 488]
[264, 231, 6, 361]
[283, 190, 20, 427]
[224, 223, 16, 293]
[295, 224, 17, 489]
[266, 228, 10, 362]
[307, 182, 31, 428]
[224, 227, 8, 294]
[294, 205, 14, 490]
[271, 228, 7, 363]
[290, 200, 28, 429]
[229, 220, 18, 295]
[308, 208, 18, 491]
[283, 161, 17, 368]
[299, 197, 31, 430]
[247, 212, 17, 296]
[285, 213, 22, 492]
[299, 163, 26, 369]
[283, 196, 30, 431]
[297, 157, 8, 370]
[298, 209, 29, 493][227, 207, 11, 298]

[301, 185, 40, 432]
[304, 155, 40, 371]
[301, 216, 21, 494]
[227, 215, 12, 299]
[296, 195, 38, 433]
[311, 159, 34, 372]
[295, 221, 24, 495]
[231, 214, 18, 300]
[293, 220, 26, 496][298, 160, 23, 374]

[312, 190, 28, 437]
[243, 210, 42, 301]
[293, 216, 40, 497]
[312, 198, 25, 439]
[293, 158, 16, 375]
[244, 204, 19, 302]
[296, 212, 8, 500]
[307, 195, 28, 440]
[315, 157, 9, 376]
[236, 199, 23, 304]
[302, 194, 22, 441]
[301, 209, 6, 502]
[314, 160, 14, 377]
[237, 188, 18, 306]
[315, 2

[270, 203, 17, 906]
[204, 281, 40, 1037]
[276, 173, 32, 855]
[254, 200, 32, 970]
[203, 275, 34, 907]
[191, 285, 37, 1038]
[275, 184, 28, 856]
[250, 207, 10, 972]
[204, 286, 16, 908]
[203, 284, 33, 1039]
[275, 183, 20, 857]
[244, 203, 14, 973]
[194, 275, 6, 910]
[207, 270, 34, 1040]
[275, 178, 22, 858]
[254, 178, 6, 974]
[197, 281, 9, 912]
[194, 283, 19, 1044]
[266, 182, 21, 859]
[253, 201, 40, 975]
[195, 280, 21, 913]
[199, 260, 43, 1045]
[264, 188, 22, 860]
[250, 206, 40, 976]
[200, 277, 28, 914]
[258, 182, 16, 861]
[203, 193, 11, 977]
[201, 273, 22, 915]
[262, 185, 18, 862]
[249, 195, 19, 978]
[202, 273, 31, 916]
[262, 177, 8, 863]
[248, 193, 21, 979]
[200, 276, 14, 917]
[263, 197, 8, 864]
[234, 183, 9, 980]
[202, 273, 26, 918]
[289, 195, 12, 865]
[226, 192, 34, 983]
[198, 267, 31, 919]
[200, 186, 16, 984]
[193, 266, 23, 920]
[196, 279, 29, 921]
[188, 279, 31, 922]
[191, 274, 29, 923]
[200, 269, 22, 924]
TdTomato
[194, 195, 7, 60]
[234, 194, 40, 189]
[230, 186, 38, 0]
[197, 194, 31, 

[293, 158, 16, 375]
[295, 221, 24, 495]
[244, 204, 19, 302][296, 195, 38, 433]

[315, 157, 9, 376]
[293, 220, 26, 496]
[236, 199, 23, 304]
[312, 190, 28, 437]
[314, 160, 14, 377]
[293, 216, 40, 497]
[296, 212, 8, 500]
[312, 198, 25, 439]
[309, 156, 25, 378]
[237, 188, 18, 306]
[301, 209, 6, 502]
[307, 195, 28, 440]
[310, 157, 20, 379]
[283, 196, 10, 309]
[315, 209, 22, 503]
[302, 194, 22, 441]
[260, 195, 14, 311]
[276, 183, 16, 380]
[309, 218, 36, 504]
[289, 182, 16, 442]
[268, 200, 14, 312]
[301, 216, 10, 507]
[310, 218, 32, 505]
[312, 245, 8, 576]
[307, 188, 20, 443]
[307, 212, 12, 508]
[310, 248, 26, 577]
[311, 216, 20, 506]
[296, 183, 25, 444]
[311, 214, 14, 509]
[302, 248, 24, 578]
[313, 193, 21, 445]
[300, 281, 16, 637]
[311, 219, 10, 510]
[243, 258, 41, 698]
[294, 243, 28, 579][298, 262, 6, 639]

[315, 221, 30, 511]
[242, 263, 31, 699]
[290, 241, 22, 580]
[283, 268, 9, 640]
[317, 219, 25, 512]
[294, 268, 8, 641]
[227, 264, 15, 700]
[301, 259, 19, 581]
[310, 223, 36, 513]
[300, 2

[196, 279, 29, 921]
[263, 197, 8, 864]
[203, 193, 11, 977]
[289, 195, 12, 865]
[188, 279, 31, 922]
[249, 195, 19, 978]
[248, 193, 21, 979]
[191, 274, 29, 923]
[234, 183, 9, 980]
[200, 269, 22, 924]
[226, 192, 34, 983]
[200, 186, 16, 984]
VGlut1


In [14]:
outArray.shape

(12, 793)

In [16]:
outArray

array([[ 2.61983471,  2.40495868,  2.20586026, ...,  2.14650639,
         2.2133734 ,  2.43951916],
       [ 5.57024793,  4.83696469,  3.00601052, ...,  3.67843727,
         3.81667919,  4.78211871],
       [ 3.92261458,  3.89256198,  2.9744553 , ...,  3.756574  ,
         3.63335838,  5.09917355],
       ...,
       [ 5.81968445,  1.38692712,  0.7768595 , ...,  1.80240421,
         3.88204358,  5.41172051],
       [43.46506386, 28.76408715,  4.21111946, ..., 20.58001503,
         9.03155522,  6.56273479],
       [ 5.8580015 ,  7.08489857,  3.21262209, ...,  3.23816679,
         3.84522915,  8.31104433]])

In [29]:
#np.savetxt("rorb_avg_at.csv", np.column_stack([ids, outArray]), delimiter=",", fmt = "%d", header = "ids, ")
#np.concatenate((trmp, outArray), axis = 2)

In [34]:
#np.hstack((np.transpose(np.asarray(ids)),outArray))

In [20]:
head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
#np.savetxt("rorb_avg_at.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)
np.savetxt("rorb_avg_at_123ids.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)


In [18]:
#np.savetxt("rorb_avg_at_orderIDS.csv", np.transpose(ids), delimiter=",", fmt = "%d", header = 'ids')
np.savetxt("rorb_avg_at_IDS123.csv", np.transpose(ids), delimiter=",", fmt = "%d", header = 'ids')

In [21]:
#np.savetxt("rorb_avg_at_orderLocations.csv", loc, delimiter=",", fmt = "%d", header = 'x,y,z')
np.savetxt("rorb_avg_at_123Locations.csv", loc, delimiter=",", fmt = "%d", header = 'x,y,z,id')

In [17]:
[idss + 1 for idss in ids]

[1,
 2,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 30,
 32,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 57,
 58,
 59,
 60,
 61,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 82,
 84,
 85,
 88,
 89,
 90,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 107,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 159,
 160,
 161,
 162,
 163,
 165,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 189,
 190,
 193,
 195,
 196,
 197,
 198,
 201,
 202,
 203,
 204,
 205,
 206,
 208,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 